# input(나이와 좋아하는 웹툰) 입력
## 옵션 1: q를 입력받을 때까지 계속 받음, 3개 이상

+ 19세 이하일 경우 성인용 웹툰 data에서 삭제
+ 입력한 웹툰 중 평점이 8.5 이하인 웹툰이 있으면 평점 8.5 이하의 웹툰도 추천, 없으면 추천하지 않음

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("./naver_daum_integration.csv")

# 나이 입력받기
age = input('나이를 입력하세요: ')

# 잘못 입력했을 경우 정확히 입력할 때까지 반복
while not age.isdigit():
    age = input('정확한 나이를 입력하세요: ')
    if age.isdigit():
        break
    
# 전체 웹툰 제목 받아오기 (input 값 비교용)
title = np.array(data['name'].tolist())

# 입력한 웹툰의 제목을 저장하는 리스트
input_list = list()

# q를 입력할 때까지 좋아하는 웹툰 제목을 입력받음, 무조건 세 개 이상 입력해야함
while 1:
    a = input('좋아하는 웹툰을 입력하세요: ')
    if a == 'q':
        if len(input_list) >= 3:
            break
        else:
            print("웹툰을 세 개 이상 입력하세요")
    elif a not in title:
        print('입력한 웹툰이 존재하지 않습니다.')
    else:
        input_list.append(a)

# 입력받은 웹툰 제목에 해당하는 다른 정보들을 저장하는 리스트, 각 요소는 Series type으로 되어있음
like_list = []

# like_list에 좋아하는 웹툰 정보를 저장
for i in input_list:
    like_list.append(data[data.name == i])

# 입력받은 나이가 19세 미만이면 19세 웹툰은 data(추천 데이터셋)에서 삭제
if age < 19:
    data = data[data.age_limit != 1]

# 추천 요소에 평점 고려
score_count = 0

# 좋아하는 웹툰 중 평점 8.5 이하인 웹툰이 있으면 평점 8.5 이하인 웹툰도 추천
# 없으면 평점 8.5 이하인 웹툰인 data에서 삭제
for i in like_list:
    if (i.score < 8.5).any():
        score_count += 1

if score_count == 0:
    data = data[data.score > 8.5 ]


### 위의 코드는 실행할 때마다 값을 입력해야므로 바로 값을 확인할 수 있도록 만든 연습 코드
 - 나이와 좋아하는 웹툰 input이 바로 들어가있음

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("./naver_daum_integration.csv")

age = 19

title = np.array(data['name'].tolist())

input_list = ['유미의 세포들', '무지개다리 파수꾼', '현세이의 보이는 라디오']

like_list = []

for i in input_list:
    like_list.append(data[data.name == i])

if age < 19:
    data = data[data.age_limit != 1]

score_count = 0

for i in like_list:
    if (i.score < 8.5).any():
        score_count += 1
        
if score_count == 0:
    data = data[data.score > 8.5]

data

,idx,name,score,writer,age_limit,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,keywords
0,1,인생존망,9.85,박태준,0,1,0,0,0,0,1,0,0,0,0,0,1,"['인생', '학교', '내가', '한번']"
1,2,독립일기,9.97,자까,0,1,0,1,0,0,0,0,0,0,0,0,0,"['혼자', '처음']"
2,3,모죠의 일지,9.97,모죠,0,1,0,0,0,0,0,0,0,0,0,0,0,"['머릿속', '세포', '유미', '생각']"
3,4,한림체육관,9.72,혜성,0,1,0,0,1,0,0,0,0,0,0,0,0,"['소신', '취향', '비밀', '성향', '내성', '내성적']"
4,5,유미의 세포들,9.93,이동건,0,1,0,0,0,0,1,0,0,0,0,0,1,"['한림', '학원', '무도', '체육관', '실천', '영하', '액션물', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2377,2378,타이밍,9.64,강풀,0,0,0,0,0,0,0,0,0,1,0,0,0,"['시간', '개념']"
2379,2380,바보,9.49,강풀,0,0,0,0,0,0,0,1,1,0,0,0,0,"['바보', '순정', '겨울', '사랑', '웃음', '가슴']"
2380,2381,데이지1,9.18,청설모,0,0,0,1,0,0,0,0,0,0,0,0,0,"['플래시', '확보', '부활', '이소', '청설모', '시리즈', '데뷔', ..."
2381,2382,아파트,9.18,강풀,0,0,0,0,0,0,0,0,0,1,0,0,0,"['풀만', '공포', '영화', '제작', '화제']"


## 옵션 2: 입력할 웹툰 개수를 먼저 입력함

In [ ]:
age = input('나이를 입력하세요: ')

likeNum = input('입력할 좋아하는 웹툰 개수를 입력하세요: ')

input_list = list()

for i in range(int(likeNum)):
    input_list.append(input('좋아하는 웹툰을 입력하세요: '))

## 추가 기능: 나이 입력하지 않았을 경우 오류 처리

In [ ]:
age = input('나이를 입력하세요: ')

while not age.isdigit():
    age = input('정확한 나이를 입력하세요: ')
    if age.isdigit():
        break
        
age